In [42]:
import json
import os
import random
import sys

import tensorflow as tf
import numpy as np
import pickle
from bert_serving.client import BertClient, ConcurrentBertClient
from tensorflow.estimator import BaselineClassifier
from tensorflow.python.estimator.canned.dnn import DNNClassifier
from tensorflow.python.estimator.run_config import RunConfig
from tensorflow.python.estimator.training import TrainSpec, EvalSpec, train_and_evaluate

tf.logging.set_verbosity(tf.logging.INFO)

In [43]:
batch_size = 128
num_parallel_calls = 1
bc = BertClient()

In [ ]:
%%time
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

encode_count = 0
def encode(chunk):
    global encode_count
    print('Chunk {}'.format(encode_count))
    encode_count += 1
    return bc.encode(chunk)
        
def get_encodes(data):
    # x is `batch_size` of lines, each of which is a json object
    features = np.array([])
    text = [x[0] for x in data]
    features = np.concatenate([encode(chunk) for chunk in chunks(text, 256)])
    # randomly choose a label
    labels = [x[1] for x in data]
    return features, labels

def cache_data(data_dir, dest_dir, start_chunk, end_chunk):
    pos_files = os.listdir(os.path.join(data_dir, 'pos'))
    neg_files = os.listdir(os.path.join(data_dir, 'neg'))
        
    data = []
    for pos_file, neg_file in zip(pos_files, neg_files):
        with open(os.path.join(data_dir, 'pos', pos_file)) as f:
            review = f.readlines()[0].strip()
            data.append((review, 1))
        with open(os.path.join(data_dir, 'neg', neg_file)) as f:
            review = f.readlines()[0].strip()
            data.append((review, 0))
    chunk_num = -1
    chunk_size = 2048
    for chunk in chunks(data, chunk_size):
        chunk_num += 1
        if chunk_num < start_chunk:
            continue
        if chunk_num > end_chunk:
            break
        features, output = get_encodes(chunk)
        print('Wrote data_{:03d}.p'.format(chunk_num))
        with open(os.path.join(dest_dir, 'data_{:03d}.p'.format(chunk_num)), 'wb') as f:
            pickle.dump((features, output), f)

input_fn_train = cache_data('/home/eugenet/final_project/data/aclImdb/train/', '/home/eugenet/final_project/cached_data/train_cased_small', 0, 14)
input_fn_eval = cache_data('/home/eugenet/final_project/data/aclImdb/test/', '/home/eugenet/final_project/cached_data/test_cased_small', 0, 14)

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Wrote data_000.p
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12
Chunk 13
Chunk 14
Chunk 15
Wrote data_001.p
Chunk 16
Chunk 17
Chunk 18
Chunk 19
Chunk 20
Chunk 21
Chunk 22
Chunk 23
Wrote data_002.p
Chunk 24
Chunk 25
Chunk 26
Chunk 27
Chunk 28
Chunk 29
Chunk 30
Chunk 31
Wrote data_003.p
Chunk 32
Chunk 33
Chunk 34
Chunk 35
Chunk 36
Chunk 37
Chunk 38
Chunk 39
Wrote data_004.p
Chunk 40
Chunk 41
Chunk 42
Chunk 43
Chunk 44
Chunk 45
Chunk 46
Chunk 47
Wrote data_005.p
Chunk 48
Chunk 49
Chunk 50
Chunk 51
Chunk 52
Chunk 53
Chunk 54
Chunk 55
Wrote data_006.p
Chunk 56
Chunk 57
Chunk 58
Chunk 59
Chunk 60
Chunk 61
Chunk 62
Chunk 63
Wrote data_007.p
Chunk 64
Chunk 65
Chunk 66
Chunk 67
Chunk 68
Chunk 69
Chunk 70
Chunk 71
Wrote data_008.p
Chunk 72
Chunk 73
Chunk 74
Chunk 75
Chunk 76
Chunk 77
Chunk 78
Chunk 79
Wrote data_009.p
Chunk 80
Chunk 81
Chunk 82
Chunk 83
Chunk 84
Chunk 85
Chunk 86
Chunk 87
Wrote data_010.p
Chunk 88
Chunk 89
Chunk 90
Chun

In [5]:
?pickle.dump